In [31]:
# prepare dataset, and print length
import torchvision
train_data = torchvision.datasets.CIFAR10("./CIFAR10_dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10("./CIFAR10_dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                          download=True)

train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"the length of train_data: {train_data_size}")
print(f"the length of test_data: {test_data_size}")


the length of train_data: 50000
the length of test_data: 10000


In [32]:
# Use DataLoader to load data
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [33]:
# Build a neral network
from torch import nn
class CIFAR10Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5 ,1, 2), # (in_channel, out_channel, kernel_size, stride, padding)
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5 ,1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5 ,1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        output = self.model(x)
        return output
    
CIFAR_model = CIFAR10Model()

In [34]:
# loss function and optimizer
import torch

loss_fn = nn.CrossEntropyLoss()

learning_rate = 0.01
optimizer = torch.optim.SGD(CIFAR_model.parameters(), lr=learning_rate)

In [ ]:
# setup parameters
total_train_step = 0   # number of training
total_test_step = 0  # number of testing
epoch = 20     # role of training

# tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("../logs_train")

# training
for i in range(epoch):
    print(f"------- Epoch {i+1} Training Started -------")

    # training start
    CIFAR_model.train()
    for data in train_dataloader:
        imgs, targets = data
        output = CIFAR_model(imgs)
        loss = loss_fn(output, targets)
        
        # backward propogation and updating parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print(f"number of training: {total_train_step}, loss: {loss.item()}") #.item() changes tensor to a number
            writer.add_scalar("train_loss", loss.item(), total_train_step) # record loss in tensorboard
    
    # testing start (after each epoch)
    CIFAR_model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():   # no gradient calculation
        for data in test_dataloader:
            imgs, targets = data
            output = CIFAR_model(imgs)
            loss = loss_fn(output, targets)
            total_test_loss += loss
            accuracy = (output.argmax(1) == targets).sum()
            total_accuracy += accuracy

    print(f"loss on the entire test set: {total_test_loss}")
    print(f"accuracy on the entire test set: {total_accuracy/test_data_size}")
    writer.add_scalar("test_loss", total_test_loss.item(), total_test_step) # record loss in tensorboard
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    # save each epoch
    torch.save(CIFAR_model, f"CIFAR_model_{i}.pth")
    print("model saved successfully")

writer.close()
    

------- Epoch 1 Training Started -------
number of training: 100, loss: 1.0236831903457642
number of training: 200, loss: 1.1650439500808716
number of training: 300, loss: 1.333216667175293
number of training: 400, loss: 1.0735360383987427
number of training: 500, loss: 1.0748335123062134
number of training: 600, loss: 1.1657850742340088
number of training: 700, loss: 1.3397513628005981
loss on the entire test set: 202.34957885742188
accuracy on the entire test set: 0.5414000153541565
model saved successfully
------- Epoch 2 Training Started -------
number of training: 800, loss: 1.0589683055877686
number of training: 900, loss: 1.092885971069336
number of training: 1000, loss: 1.1265993118286133
number of training: 1100, loss: 1.263832926750183
number of training: 1200, loss: 1.1347509622573853
number of training: 1300, loss: 1.0229456424713135
number of training: 1400, loss: 1.010515809059143
number of training: 1500, loss: 1.0808587074279785
loss on the entire test set: 193.68362426

KeyboardInterrupt: 